In [19]:
import cv2
import numpy as np
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
class general_pose_model(object):
    def __init__(self, modelpath):
        # Specify the model to be used
        #   Body25: 25 points
        #   COCO:   18 points
        #   MPI:    15 points
        self.threshold = 0.1
        self.pose_net = self.general_coco_model(modelpath)

    def general_coco_model(self, modelpath):
        self.points_name = {
            "Nose": 0, "Neck": 1, 
            "RShoulder": 2, "RElbow": 3, "RWrist": 4,
            "LShoulder": 5, "LElbow": 6, "LWrist": 7, 
            "RHip": 8, "RKnee": 9, "RAnkle": 10, 
            "LHip": 11, "LKnee": 12, "LAnkle": 13, 
            "REye": 14, "LEye": 15, 
            "REar": 16, "LEar": 17, 
            "Background": 18}
        self.num_points = 18
        self.point_pairs = [[1, 0], [1, 2], [1, 5], 
                            [2, 3], [3, 4], [5, 6], 
                            [6, 7], [1, 8], [8, 9],
                            [9, 10], [1, 11], [11, 12], 
                            [12, 13], [0, 14], [0, 15], 
                            [14, 16], [15, 17], [2, 17], [5, 16]]
        
        self.mapIdx = [[31,32], [39,40], [33,34], [35,36], [41,42], [43,44], 
                  [19,20], [21,22], [23,24], [25,26], [27,28], [29,30], 
                  [47,48], [49,50], [53,54], [51,52], [55,56], 
                  [37,38], [45,46]]
        
        self.colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
                         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
                         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]
        
        prototxt   = os.path.join(
            modelpath, 
            'pose_deploy_linevec.prototxt.txt')
        caffemodel = os.path.join(
            modelpath, 
            'pose_iter_440000.caffemodel')
        coco_model = cv2.dnn.readNetFromCaffe(prototxt, caffemodel)

        return coco_model

    def predict(self, imgfile):
        img_cv2 = cv2.imread(imgfile)
        img_height, img_width, _ = img_cv2.shape
        
        inHeight = 368
        inWidth = int((inHeight/img_height)*img_width)

        
        inpBlob = cv2.dnn.blobFromImage(img_cv2, 
                                        1.0 / 255, 
                                        (inWidth, inHeight),
                                        (127.5, 127.5, 127.5), 
                                        swapRB=False, 
                                        crop=False)
        self.pose_net.setInput(inpBlob)
        self.pose_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
        self.pose_net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)

        output = self.pose_net.forward()
        
        points = []
        for idx in range(self.num_points):
            probMap = output[0, idx, :, :] 
            probMap = cv2.resize(probMap, (img_cv2.shape[1], img_cv2.shape[0]))
            mapSmooth = cv2.GaussianBlur(probMap,(3,3),0,0)
            mapMask = np.uint8(mapSmooth > self.threshold)
            
            contours, _ = cv2.findContours(mapMask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            keypoints = []
            for cnt in contours:
                blobMask = np.zeros(mapMask.shape)
                blobMask = cv2.fillConvexPoly(blobMask, cnt, 1)
                maskedProbMap = mapSmooth * blobMask
                _, maxVal, _, maxLoc = cv2.minMaxLoc(maskedProbMap)
        
                x = maxLoc[0]
                y = maxLoc[1]
                prob = probMap[maxLoc[1], maxLoc[0]]

                keypoints.append(x)
                keypoints.append(y)
                keypoints.append(prob)
            
            if len(keypoints) == 3:
                points.append(keypoints[0])
                points.append(keypoints[1])
                points.append(keypoints[2])
            else:
                points.append(0)
                points.append(0)
                points.append(0)

        return points

def generate_pose_keypoints(img_file):

    modelpath = os.getcwd()
    pose_model = general_pose_model(modelpath)

    res_points = pose_model.predict(img_file)
    
    pose_data = {"version": 1,
                 "people":  [
                                {"pose_keypoints": res_points}
                            ]
                }

    pose_keypoints_path = img_file.replace('.jpg', '_keypoints.json')

    json_object = json.dumps(pose_data, indent = 4) 
  
    # Writing to sample.json 
    with open(pose_keypoints_path, "w") as outfile: 
        outfile.write(json_object) 
    print('File saved at {}'.format(pose_keypoints_path))